In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import torch

# 프로젝트 모듈
from preprocessing.vectorization.bert_vectorizer import BERTVectorizer
from preprocessing.vectorization.loader import load_files

# 설정
DRAMA_FILE_PATH = "files/final_files/drama/00_drama_main.parquet"
MOVIE_FILE_PATH = "files/final_files/movie/00_movie_main.parquet"
REVIEW_FILE_PATH = "files/review_all.parquet"
HIT_FILE_PATH = "files/final_files/00_hit_score.parquet"

OUTPUT_DIR = "files/embeddings"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

파일 로드

In [ ]:
# 드라마 데이터
try:
    drama_data = load_files(
        main_file_path=DRAMA_FILE_PATH,
        hit_file_path=HIT_FILE_PATH,
        threshold=150  # 줄거리 최소 길이
    )
    print(f"드라마 데이터: {len(drama_data):,}개")
    print(f"   - 컬럼: {list(drama_data.columns)}")
except Exception as e:
    print(f"드라마 데이터 로드 실패: {e}")
    drama_data = None

print()

# 영화 데이터
try:
    movie_data = load_files(
        main_file_path=MOVIE_FILE_PATH,
        hit_file_path=HIT_FILE_PATH,
        threshold=150
    )
    print(f"영화 데이터: {len(movie_data):,}개")
    print(f"   - 컬럼: {list(movie_data.columns)}")
except Exception as e:
    print(f"영화 데이터 로드 실패: {e}")
    movie_data = None

# 리뷰 데이터
try:
    review_data = pd.read_parquet(REVIEW_FILE_PATH)

    # 리뷰 텍스트 길이 필터링 (너무 짧은 리뷰 제외)
    review_data = review_data[review_data['review_text_clean'].str.len() >= 50].copy()

    print(f"리뷰 데이터: {len(review_data):,}개")
    print(f"   - 컬럼: {list(review_data.columns)}")
    print(f"   - 리뷰 텍스트 길이: {review_data['review_text_clean'].str.len().describe()}")
except Exception as e:
    print(f"리뷰 데이터 로드 실패: {e}")
    review_data = None

드라마 데이터: 2,334개
   - 컬럼: ['imdb_id', 'overview', 'hit_score', 'len_overview', 'hit_label', 'nonhit_label']

영화 데이터: 20,551개
   - 컬럼: ['imdb_id', 'overview', 'hit_score', 'len_overview', 'hit_label', 'nonhit_label']


드라마 임베딩

In [ ]:
if drama_data is not None:
    print("="*80)
    print("🎬 드라마 임베딩 생성 시작")
    print("="*80)

    # Vectorizer 초기화
    drama_vectorizer = BERTVectorizer(type='drama')

    # 임베딩 생성 (청크 단위)
    drama_embeddings = drama_vectorizer.fit_transform(
        data=drama_data,
        columns=['overview'],  # 줄거리만 사용
        use_chunks=True  # 대용량 데이터 처리
    )

    print(f"\n드라마 임베딩 생성 완료: {len(drama_embeddings):,}개")
    print(f"   - 임베딩 차원: {len(drama_embeddings['embedding'].iloc[0])}")

    # 저장
    output_path = f"{OUTPUT_DIR}/drama_embeddings.parquet"
    drama_embeddings.to_parquet(output_path, index=False)
    print(f"저장 완료: {output_path}")

else:
    print("⚠드라마 데이터가 없어 임베딩 생성을 건너뜁니다.")

영화 임베딩

In [ ]:
if movie_data is not None:
    print("\n" + "="*80)
    print("🎥 영화 임베딩 생성 시작")
    print("="*80)

    # Vectorizer 초기화
    movie_vectorizer = BERTVectorizer(type='movie')

    # 임베딩 생성 (청크 단위)
    movie_embeddings = movie_vectorizer.fit_transform(
        data=movie_data,
        columns=['overview'],  # 줄거리만 사용
        use_chunk=True
    )

    print(f"\n영화 임베딩 생성 완료: {len(movie_embeddings):,}개")
    print(f"   - 임베딩 차원: {len(movie_embeddings['embedding'].iloc[0])}")

    # 저장
    output_path = f"{OUTPUT_DIR}/movie_embeddings.parquet"
    movie_embeddings.to_parquet(output_path, index=False)
    print(f"저장 완료: {output_path}")

else:
    print("영화 데이터가 없어 임베딩 생성을 건너뜁니다.")

리뷰 임베딩

In [ ]:
if review_data is not None:
    print("\n" + "="*80)
    print("리뷰 임베딩 생성 시작")
    print("="*80)

    # Vectorizer 초기화
    review_vectorizer = BERTVectorizer(type='review')

    # 임베딩 생성 (청크 단위)
    # 리뷰는 데이터가 많을 수 있으므로 청크 처리 필수!
    review_embeddings = review_vectorizer.fit_transform(
        data=review_data,
        columns=['review_text_clean'],  # 정제된 리뷰 텍스트
        use_chunk=True  # 대용량 데이터 처리
    )

    print(f"\n리뷰 임베딩 생성 완료: {len(review_embeddings):,}개")
    print(f"   - 임베딩 차원: {len(review_embeddings['embedding'].iloc[0])}")

    # 저장
    output_path = f"{OUTPUT_DIR}/review_embeddings.parquet"
    review_embeddings.to_parquet(output_path, index=False)
    print(f"저장 완료: {output_path}")

else:
    print("리뷰 데이터가 없어 임베딩 생성을 건너뜁니다.")